# Proyecto Sprint 10: Aprendizaje Supervisado

## Introducción

Para este proyecto, tenemos el planteamiento que los clientes de Beta Bank se están yendo, cada mes, poco a poco. Los banqueros descubrieron que es más barato salvar a los clientes existentes que atraer nuevos.

Necesitamos predecir si un cliente dejará el banco pronto. Se tiene a disposición los datos sobre el comportamiento pasado de los clientes y la terminación de contratos con el banco, lo cuál nos será de utilidad para determinar lo anterior.



## Inicialización

Iniciamos con la importación de las librerías requeridas para el proyecto.

In [111]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score

## Carga de Datos

Continuamos con la lectura de dataset, revisando que el separador por defecto sea efectivamente el que necesitamos. Para ello, imprimimos una muestra.

In [2]:
df = pd.read_csv("/datasets/Churn.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [3]:
df.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


Una primera mirada a los datos nos indican que de forma predeterminada viene separado por comas. Adicionalmente, mediante el metodo info vemos las columnas que posee y sus tipos. Vemos que sus columnas están con Mayúsculas en sus iniciales, lo que cambiaremos a posterior. También, inicialmente vemos que de todas las columnas, solo "Tenure" posee datos faltantes.

Por otra parte, mediante el metodo describe obtenemos la visión general de los datos para sus respectivas estadísticas.

## Preparación de los datos



### Cambio a minúsculas

Comenzaremos eliminando las Mayúsculas de las columnas.

In [4]:
df.columns = df.columns.str.lower() #señalamos que transforme todas las columnas a minusculas
df.sample()

,rownumber,customerid,surname,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited
4382,4383,15780561,Hay,622,France,Female,39,NaN,83456.79,2,0,0,38882.34,0


### Tratado de valores ausentes y duplicados

Debemos cerciorarnos que cada una de las columnas tenga coherencia en sus datos. Para ello, verificaremos sus duplicados y valores ausentes.

In [5]:
df.isna().sum()

rownumber            0
customerid           0
surname              0
creditscore          0
geography            0
gender               0
age                  0
tenure             909
balance              0
numofproducts        0
hascrcard            0
isactivemember       0
estimatedsalary      0
exited               0
dtype: int64

In [6]:
df[df["tenure"].isna()]

,rownumber,customerid,surname,creditscore,geography,gender,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited
30,31,15589475,Azikiwe,591,Spain,Female,39,NaN,0.00,3,1,0,140469.38,1
48,49,15766205,Yin,550,Germany,Male,38,NaN,103391.38,1,0,1,90878.13,0
51,52,15768193,Trevisani,585,Germany,Male,36,NaN,146050.97,2,0,0,86424.57,0
53,54,15702298,Parkhill,655,Germany,Male,41,NaN,125561.97,1,0,0,164040.94,1
60,61,15651280,Hunter,742,Germany,Male,35,NaN,136857.00,1,0,0,84509.57,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9944,9945,15703923,Cameron,744,Germany,Male,41,NaN,190409.34,2,1,1,138361.48,0
9956,9957,15707861,Nucci,520,France,Female,46,NaN,85216.61,1,1,0,117369.52,1
9964,9965,15642785,Douglas,479,France,Male,34,NaN,117593.48,2,0,0,113308.29,0
9985,9986,15586914,Nepean,659,France,Male,36,NaN,123841.49,2,1,0,96833.00,0


In [7]:
df.duplicated().sum()

0

No contaríamos con filas exactamente iguales en nuestro dataset.

Para el tratamiento de valores ausentes, tenemos:

In [8]:
print(df["tenure"].value_counts(dropna = False))
print("la media es:",df["tenure"].mean())
print("la mediana es:",df["tenure"].median())

1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
NaN     909
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: tenure, dtype: int64
la media es: 4.997690023099769
la mediana es: 5.0


Ya que la media y mediana son practicamente iguales, procederemos a rellenar los valures ausentes de la columna Tenure con el valor 5.

In [9]:
df["tenure"].fillna(5, inplace=True)
print(df["tenure"].isna().sum())
print(df["tenure"].value_counts(dropna = False))

0
5.0     1836
1.0      952
2.0      950
8.0      933
3.0      928
7.0      925
4.0      885
9.0      882
6.0      881
10.0     446
0.0      382
Name: tenure, dtype: int64


### Codificación de características categóricas

Ahora, podemos codificar la columna "gender" para que tambien analice aquella categoría.

In [10]:
df["gender"].value_counts()

Male      5457
Female    4543
Name: gender, dtype: int64

Ya que solo tenemos dos categorías (Male, Female), en vez de codificar, haremos solo un mapeo por medio de pandas:

In [11]:
gender_dict = {'Male': 0, 'Female': 1}
df['gender'] = df['gender'].map(gender_dict)

In [12]:
df["gender"].value_counts()

0    5457
1    4543
Name: gender, dtype: int64

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rownumber        10000 non-null  int64  
 1   customerid       10000 non-null  int64  
 2   surname          10000 non-null  object 
 3   creditscore      10000 non-null  int64  
 4   geography        10000 non-null  object 
 5   gender           10000 non-null  int64  
 6   age              10000 non-null  int64  
 7   tenure           10000 non-null  float64
 8   balance          10000 non-null  float64
 9   numofproducts    10000 non-null  int64  
 10  hascrcard        10000 non-null  int64  
 11  isactivemember   10000 non-null  int64  
 12  estimatedsalary  10000 non-null  float64
 13  exited           10000 non-null  int64  
dtypes: float64(3), int64(9), object(2)
memory usage: 1.1+ MB


De esta manera, queda el hombre con un valor de 0 y mujer con un valor de 1.


Con esto, tendríamos nuestros datos casi listos, solo debemos segmentar y estandarizar nuestro dataset.

### Segmentación del Dataset

Para el dataset de entrenamiento dejaremos 60% del dataframe total, para el de validación y prueba un 20%

In [14]:
#generamos una funcion para dividir nuestro dataset dos veces usando la funcion train_test_split

def split_train_val_test(df_input, stratify_colname='y', frac_train=0.6, frac_val=0.2, frac_test=0.2, 
                         random_state=12345):
    

    if frac_train + frac_val + frac_test != 1.0: #verifica que la suma de las particiones de 1
        raise ValueError('Los elementos %f, %f, %f no suman 1.0' % \
                         (frac_train, frac_val, frac_test))

    if stratify_colname not in df_input.columns:
        raise ValueError('%s no es una columna en el dataframe' % (stratify_colname))

    X = df_input # Posee todas las columnas
    y = df_input[[stratify_colname]] # la columna en la cual basamos el analisis

    # Divide el dataframe en porciones temporales
    df_train, df_temp, y_train, y_temp = train_test_split(X, y, stratify=y, test_size=(1.0 - frac_train),
                                                          random_state=random_state)

    # Divide finalmente el dataframe utilizando las porciones temporales para obtener las 3 partes
    relative_frac_test = frac_test / (frac_val + frac_test)
    df_val, df_test, y_val, y_test = train_test_split(df_temp, y_temp, stratify=y_temp, 
                                                      test_size=relative_frac_test, random_state=random_state)

    assert len(df_input) == len(df_train) + len(df_val) + len(df_test) #Verifica la integridad de los datos

    return df_train, df_val, df_test

Utilizando nuestra funcion para crear los 3 dataset:

In [15]:
df_train, df_val, df_test = split_train_val_test(df, stratify_colname='exited', 
                                                 frac_train=0.60, frac_val=0.20, frac_test=0.20)

Evaluando la integridad de los datos, inicialmente vimos que teníamos 10000 filas, esto implica que nuestro conjunto de entrenamiento debe contener el 60% de los datos o 6000 filas y los de validación y prueba 2000 filas cada uno. Verificando esto, tenemos:

In [16]:
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(6000, 14)
(2000, 14)
(2000, 14)


Ahora, para crear nuestros dataset de entrenamiento, validación y prueba, efectuamos lo siguiente:

In [17]:
features_train = df_train.drop("exited", axis = 1)
target_train = df_train["exited"]
features_valid = df_val.drop("exited", axis = 1)
target_valid = df_val["exited"]
features_test = df_test.drop("exited", axis = 1)
target_test = df_test["exited"]

### Estandarización de Datos

A pesar de que nuestro dataset tiene datos que son numéricos, creo que no son relevantes para señalarlos dentro del modelo. Por ejemplo la columna "rownumber" y el "customerid" no deberían tener ninguna implicancia en determinar si los clientes puedan o no dejar el banco, por lo que quedarán fuera de nuestras columnas "numéricas".

In [18]:
numeric = ['creditscore', 'gender', 'age', 'tenure', 'balance', 'numofproducts', 'hascrcard', 'isactivemember',
           'estimatedsalary']

In [19]:
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

Con esto efectuado podemos ya comenzar nuestros modelos.

## Modelos de ML

### Modelo Árbol de Decisión

In [20]:
tree_score = 0
best_depth = 0
for a in range(1,11):
    model = DecisionTreeClassifier(random_state=1702, max_depth = a)
    model.fit(features_train[numeric], target_train)
    score = model.score(features_train[numeric], target_train)
    valid_score = model.score(features_valid[numeric], target_valid)
    predicted_valid = model.predict(features_valid[numeric])
    if valid_score > tree_score:
        tree_score = valid_score
        best_depth = a
print("Profundidad =", best_depth, ":", tree_score,"; Calidad conjunto entrenamiento:", score)

Profundidad = 5 : 0.8575 ; Calidad conjunto entrenamiento: 0.8988333333333334


Ahora, para revisar su equilibrio de clases veremos su recall y precision score:

In [21]:
recall_score(target_valid, predicted_valid)

0.44607843137254904

In [22]:
precision_score(target_valid, predicted_valid)

0.6523297491039427

Los valores nos señalan que las métricas están bien lejos de lo esperado. Veremos ahora el valor F1 para determinar cómo proceder:

In [23]:
f1_score(target_valid,predicted_valid)

0.529839883551674

De forma inicial, nos solicitan que para nuestro modelo tenga al menos un valor de F1 de 0.59, por lo que este no calificaría, pero aun podemos mejorarlo equilibrando las clases.

#### Ajuste de Desequilibrio de Clases

Para ajustar el desequilibrio de clases partiremos modificando el peso de las mismas.

**Ajuste de Peso de Clase**

In [24]:
tree_score = 0
best_depth = 0
for a in range(1,100):
    model = DecisionTreeClassifier(random_state=1702, max_depth = a, class_weight = 'balanced')
    model.fit(features_train[numeric], target_train)
    score = model.score(features_train[numeric], target_train)
    valid_score = model.score(features_valid[numeric], target_valid)
    predicted_valid = model.predict(features_valid[numeric])
    if valid_score > tree_score:
        tree_score = valid_score
        best_depth = a
print("Profundidad =", best_depth, ":", tree_score,"; Calidad conjunto entrenamiento:", score)

Profundidad = 3 : 0.7975 ; Calidad conjunto entrenamiento: 1.0


In [25]:
print("F1 Score:",f1_score(target_valid,predicted_valid))
print("Precision Score:", precision_score(target_valid,predicted_valid))
print("Recall Score:", recall_score(target_valid,predicted_valid))

F1 Score: 0.47897196261682246
Precision Score: 0.4575892857142857
Recall Score: 0.5024509803921569


La calidad del conjunto de entrenamiento quedó con el valor máximo. Sin embargo, la calidad del conjunto de validación bajó. Esto también se refleja en el valor de F1 que baja respecto de lo anterior.

**Sobremuestreo**

Generamos una función para obtener el sobremuestreo:

In [102]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
    return features_upsampled, target_upsampled


features_upsampled, target_upsampled = upsample(features_train[numeric], target_train, 40)

Entrenamos el modelo nuevamente:

In [103]:
for a in range(1,100):
    model = DecisionTreeClassifier(random_state=1702, max_depth = a, class_weight = 'balanced')
    model.fit(features_upsampled, target_upsampled)
    score = model.score(features_train[numeric], target_train)
    valid_score = model.score(features_valid[numeric], target_valid)
    predicted_valid = model.predict(features_valid[numeric])
    if valid_score > tree_score:
        tree_score = valid_score
        best_depth = a
print("Profundidad =", best_depth, ":", tree_score,"; Calidad conjunto entrenamiento:", score)

print('F1:', f1_score(target_valid, predicted_valid))

Profundidad = 5 : 0.804 ; Calidad conjunto entrenamiento: 1.0
F1: 0.48000000000000004


**Submuestreo**

Generamos una función para obtener el submuestreo:

In [96]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=1702)]+[features_ones])
    target_downsampled = pd.concat([target_zeros.sample(frac=fraction, random_state=1702)]+[target_ones])

    features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=1702)
    return features_downsampled, target_downsampled


features_downsampled, target_downsampled = downsample(features_train[numeric], target_train, 0.6)

Entrenamos el modelo bajo estos parámetros:

In [97]:
for a in range(1,100):
    model = DecisionTreeClassifier(random_state=1702, max_depth = a, class_weight = 'balanced')
    model.fit(features_downsampled, target_downsampled)
    score = model.score(features_train[numeric], target_train)
    valid_score = model.score(features_valid[numeric], target_valid)
    predicted_valid = model.predict(features_valid[numeric])
    if valid_score > tree_score:
        tree_score = valid_score
        best_depth = a
print("Profundidad =", best_depth, ":", tree_score,"; Calidad conjunto entrenamiento:", score)

print('F1:', f1_score(target_valid, predicted_valid))

Profundidad = 5 : 0.804 ; Calidad conjunto entrenamiento: 0.9378333333333333
F1: 0.4841675178753831


**Conclusión**

Para nuestro modelo de Árbol de Decisión, considerando las mejoras en los desequilibrios de clases, primero que todo, dejamos el peso de las clases como "balanceado", eso genera que el dataset de entrenamiento tenga un 100% de exactitud, pero la calidad del dataset de validación baja. Posteriormente, viendo tanto el sobremuestreo como submuestreo, no logramos mejorar la calidad del factor F1 de forma considerable; de hecho baja respecto de nuestros valores iniciales (antes de equilibrar las clases).

Con el peso de las clases ajustado, el mejor valor de F1 lo entrega el submuestreo.

### Modelo Bosque Aleatorio

In [30]:
best_score = 0
best_est = 0

for est in range(1, 100): # selecciona el rango del hiperparámetro
    model2 = RandomForestClassifier(random_state=1702, n_estimators=est) # configura el número de árboles
    model2.fit(features_train[numeric], target_train) # entrena el modelo en el conjunto de entrenamiento
    score2 = model2.score(features_train[numeric], target_train)
    valid_score2 = model2.score(features_valid[numeric], target_valid)
    predicted_valid2 = model2.predict(features_valid[numeric])
    if valid_score2 > best_score:
        best_score = valid_score2 #guarda el mejor puntaje o calidad
        best_est = est # guarda el número de estimadores que corresponden a la mejor puntuación de exactitud
       
print("La exactitud en conjunto de validación es (n_estimators = ", best_est,"):", best_score)
print("La calidad del conjunto de entrenamiento es: ", score2)

La exactitud en conjunto de validación es (n_estimators =  46 ): 0.861
La calidad del conjunto de entrenamiento es:  1.0


Para el Recall y Precision tenemos:

In [31]:
print("Precision Score:", precision_score(target_valid,predicted_valid2))
print("Recall Score:", recall_score(target_valid,predicted_valid2))
print("F1 Score:",f1_score(target_valid,predicted_valid2))

Precision Score: 0.748
Recall Score: 0.4583333333333333
F1 Score: 0.5683890577507599


Este modelo se ve mas promisorio. Tenemos un valor de precisión bastante decente. Sin embargo el de recall es muy bajo. Respecto al valor de F1 se encuentra bastante cerca de lo que requerimos, por lo que ajustaremos el desequilibrio de clases para ver su mejoría.

#### Ajuste de desequilibrio de clases

**Ajuste de peso de clase**

In [32]:
best_score = 0
best_est = 0
for est in range(1, 100): # selecciona el rango del hiperparámetro
    model2 = RandomForestClassifier(random_state=1702, n_estimators=est, class_weight= 'balanced')
    model2.fit(features_train[numeric], target_train) # entrena el modelo en el conjunto de entrenamiento
    score2 = model2.score(features_train[numeric], target_train)
    valid_score2 = model2.score(features_valid[numeric], target_valid)
    predicted_valid2 = model2.predict(features_valid[numeric])
    if valid_score2 > best_score:
        best_score = valid_score2 #guarda el mejor puntaje o calidad
        best_est = est # guarda el número de estimadores que corresponden a la mejor puntuación de exactitud

print("La exactitud en conjunto de validación es (n_estimators = ", best_est,"):", best_score)
print("La calidad del conjunto de entrenamiento es: ", score2)

La exactitud en conjunto de validación es (n_estimators =  54 ): 0.859
La calidad del conjunto de entrenamiento es:  1.0


In [33]:
print("Precision Score:", precision_score(target_valid,predicted_valid2))
print("Recall Score:", recall_score(target_valid,predicted_valid2))
print("F1 Score:",f1_score(target_valid,predicted_valid2))

Precision Score: 0.7662337662337663
Recall Score: 0.4338235294117647
F1 Score: 0.5539906103286386


Ajustando el peso de las clases, el valor de Precisión mejoró, el de Recall bajó y F1 también disminuyó ligeramente.

Veremos ahora con las técnicas de Sobremuestreo y Submuestreo si logramos mejorar el valor de F1 al solicitado.

**Sobremuestreo**

In [117]:
features_upsampled2, target_upsampled2 = upsample(features_train[numeric], target_train, 10)
best_score = 0
best_est = 0
for est in range(1, 100): # selecciona el rango del hiperparámetro
    model2 = RandomForestClassifier(random_state=1702, n_estimators=est, class_weight= 'balanced')
    model2.fit(features_upsampled2, target_upsampled2)
    score2 = model2.score(features_train[numeric], target_train)
    valid_score2 = model2.score(features_valid[numeric], target_valid)
    predicted_valid2 = model2.predict(features_valid[numeric])
    if valid_score2 > best_score:
        best_score = valid_score2 #guarda el mejor puntaje o calidad
        best_est = est # guarda el número de estimadores que corresponden a la mejor puntuación de exactitud

print("La exactitud en conjunto de validación es (n_estimators = ", best_est,"):", best_score)
print("La calidad del conjunto de entrenamiento es: ", score2)

La exactitud en conjunto de validación es (n_estimators =  70 ): 0.8445
La calidad del conjunto de entrenamiento es:  0.9998333333333334


In [118]:
print("Precision Score:", precision_score(target_valid,predicted_valid2))
print("Recall Score:", recall_score(target_valid,predicted_valid2))
print("F1 Score:",f1_score(target_valid,predicted_valid2))

Precision Score: 0.6355421686746988
Recall Score: 0.5171568627450981
F1 Score: 0.5702702702702703


Podemos ver que mejoró el valor de F1, por lo que el sobremuestreo sirvió.

**Submuestreo**

In [54]:
features_downsampled2, target_downsampled2 = downsample(features_train[numeric], target_train, 0.599)
best_score = 0
best_est = 0
for est in range(1, 100): # selecciona el rango del hiperparámetro
    model2 = RandomForestClassifier(random_state=1702, n_estimators=est, class_weight= 'balanced')
    model2.fit(features_downsampled2, target_downsampled2)
    score2 = model2.score(features_train[numeric], target_train)
    valid_score2 = model2.score(features_valid[numeric], target_valid)
    predicted_valid2 = model2.predict(features_valid[numeric])
    if valid_score2 > best_score:
        best_score = valid_score2 #guarda el mejor puntaje o calidad
        best_est = est # guarda el número de estimadores que corresponden a la mejor puntuación de exactitud

print("La exactitud en conjunto de validación es (n_estimators = ", best_est,"):", best_score)
print("La calidad del conjunto de entrenamiento es: ", score2)

La exactitud en conjunto de validación es (n_estimators =  46 ): 0.8545
La calidad del conjunto de entrenamiento es:  0.9791666666666666


In [55]:
print("Precision Score:", precision_score(target_valid,predicted_valid2))
print("Recall Score:", recall_score(target_valid,predicted_valid2))
print("F1 Score:",f1_score(target_valid,predicted_valid2))

Precision Score: 0.6530612244897959
Recall Score: 0.5490196078431373
F1 Score: 0.5965379494007989


Con este resultado, tenemos un posible candidato. Para esto el submuestreo hubo que dejarlo con una partición de 0.599 para que el resultado de F1 estuviera dentro del rango solicitado.

**Conclusión**

Hasta ahora el mejor modelo ha sido el Bosque Aleatorio con la mejora de ajuste de peso y con la técnica de submuestreo para mejorar el valor de F1. Previo a efectuar cualquier mejora y ajuste ya mostraba indicios de lo mismo, ya que su valor de F1 estaba en 0.57.

### Modelo Regresión Logística

In [65]:
model3 = LogisticRegression(random_state=1702, solver= 'liblinear')
model3.fit(features_train[numeric], target_train) # entrena el modelo en el conjunto de entrenamiento
predicted_valid3 = model3.predict(features_valid[numeric])
score_train = model3.score(features_train[numeric], target_train) # calcula la exactitud en el conjunto de entrenamiento
score_valid = model3.score(features_valid[numeric], target_valid)

print("Exactitud del modelo en el conjunto de entrenamiento:", score_train)
print("Exactitud del modelo en el conjunto de validación:", score_valid)

Exactitud del modelo en el conjunto de entrenamiento: 0.8115
Exactitud del modelo en el conjunto de validación: 0.803


In [64]:
print("Precision Score:", precision_score(target_valid,predicted_valid3))
print("Recall Score:", recall_score(target_valid,predicted_valid3))
print("F1 Score:",f1_score(target_valid,predicted_valid3))

Precision Score: 0.5614035087719298
Recall Score: 0.1568627450980392
F1 Score: 0.24521072796934867


Hasta el momento, sin el equilibrio de clases, este es el modelo con el peor F1.

#### Ajuste de desequilibrio de clases

**Ajuste de peso de clase**


In [66]:
model3 = LogisticRegression(random_state=1702, solver= 'liblinear', class_weight = 'balanced')
model3.fit(features_train[numeric], target_train) # entrena el modelo en el conjunto de entrenamiento
predicted_valid3 = model3.predict(features_valid[numeric])
score_train = model3.score(features_train[numeric], target_train) # calcula la exactitud en el conjunto de entrenamiento
score_valid = model3.score(features_valid[numeric], target_valid)

print("Exactitud del modelo en el conjunto de entrenamiento:", score_train)
print("Exactitud del modelo en el conjunto de validación:", score_valid)

Exactitud del modelo en el conjunto de entrenamiento: 0.6995
Exactitud del modelo en el conjunto de validación: 0.708


In [67]:
print("Precision Score:", precision_score(target_valid,predicted_valid3))
print("Recall Score:", recall_score(target_valid,predicted_valid3))
print("F1 Score:",f1_score(target_valid,predicted_valid3))

Precision Score: 0.3839050131926121
Recall Score: 0.7132352941176471
F1 Score: 0.49914236706689535


Con el ajuste de peso de clase, el valor de F1 mejora considerablemente.

**Sobremuestreo**

In [80]:
features_upsampled3, target_upsampled3 = upsample(features_train[numeric], target_train, 10)
model3 = LogisticRegression(random_state=1702, solver= 'liblinear', class_weight = 'balanced')
model3.fit(features_upsampled3, target_upsampled3) # entrena el modelo
predicted_valid3 = model3.predict(features_valid[numeric])
score_train = model3.score(features_train[numeric], target_train) 
score_valid = model3.score(features_valid[numeric], target_valid)

print("Exactitud del modelo en el conjunto de entrenamiento:", score_train)
print("Exactitud del modelo en el conjunto de validación:", score_valid)

Exactitud del modelo en el conjunto de entrenamiento: 0.6995
Exactitud del modelo en el conjunto de validación: 0.708


In [81]:
print("Precision Score:", precision_score(target_valid,predicted_valid3))
print("Recall Score:", recall_score(target_valid,predicted_valid3))
print("F1 Score:",f1_score(target_valid,predicted_valid3))

Precision Score: 0.3839050131926121
Recall Score: 0.7132352941176471
F1 Score: 0.49914236706689535


**Submuestreo**

In [90]:
features_downsampled3, target_downsampled3 = downsample(features_train[numeric], target_train, 0.6)
model3 = LogisticRegression(random_state=1702, solver= 'liblinear', class_weight = 'balanced')
model3.fit(features_downsampled3, target_downsampled3) # entrena el modelo
predicted_valid3 = model3.predict(features_valid[numeric])
score_train = model3.score(features_train[numeric], target_train) 
score_valid = model3.score(features_valid[numeric], target_valid)

print("Exactitud del modelo en el conjunto de entrenamiento:", score_train)
print("Exactitud del modelo en el conjunto de validación:", score_valid)

Exactitud del modelo en el conjunto de entrenamiento: 0.6986666666666667
Exactitud del modelo en el conjunto de validación: 0.7105


In [91]:
print("Precision Score:", precision_score(target_valid,predicted_valid3))
print("Recall Score:", recall_score(target_valid,predicted_valid3))
print("F1 Score:",f1_score(target_valid,predicted_valid3))

Precision Score: 0.38645418326693226
Recall Score: 0.7132352941176471
F1 Score: 0.5012919896640827


**Conclusión**

Este modelo inicialmente mostró indicios de ser el más bajo. Con un valor de F1 de 0.24 se logró aumentar hasta 0.5, que, sin embargo, no logra superar al bosque aleatorio, pero sí al árbol de decisión.

### Elección del Modelo

Como bien ya señalamos previamente, el candidato seleccionado para nuestro modelo final sería el de Bosque Aleatorio con ajuste de peso y submuestreo.

#### Modelo Bosque Aleatorio Ajustado

In [112]:
features_downsampled2, target_downsampled2 = downsample(features_train[numeric], target_train, 0.599)
best_score = 0
best_est = 0
for est in range(1, 100): # selecciona el rango del hiperparámetro
    model2 = RandomForestClassifier(random_state=1702, n_estimators=est, class_weight= 'balanced')
    model2.fit(features_downsampled2, target_downsampled2)
    score2 = model2.score(features_train[numeric], target_train)
    valid_score2 = model2.score(features_valid[numeric], target_valid)
    predicted_valid2 = model2.predict(features_valid[numeric])
    predicted_test = model2.predict(features_test[numeric])
    probabilities_valid = model2.predict_proba(features_valid[numeric])
    if valid_score2 > best_score:
        best_score = valid_score2 #guarda el mejor puntaje o calidad
        best_est = est # guarda el número de estimadores que corresponden a la mejor puntuación de exactitud

print("La exactitud en conjunto de validación es (n_estimators = ", best_est,"):", best_score)
print("La calidad del conjunto de entrenamiento es: ", score2)

La exactitud en conjunto de validación es (n_estimators =  46 ): 0.8545
La calidad del conjunto de entrenamiento es:  0.9791666666666666


In [113]:
print("Precision Score:", precision_score(target_valid,predicted_valid2))
print("Recall Score:", recall_score(target_valid,predicted_valid2))
print("F1 Score:",f1_score(target_valid,predicted_valid2))

Precision Score: 0.6530612244897959
Recall Score: 0.5490196078431373
F1 Score: 0.5965379494007989


Como bien vemos, su valor de F1 está en 0.5965 lo cual podría considerarse 0.6

#### Prueba de valor F1 en dataset de prueba

In [115]:
print("F1 Score:",f1_score(target_test,predicted_test))

F1 Score: 0.5698478561549101


El valor de F1 para nuestro conjunto de prueba baja un poco respecto del de validación.

#### Valor AUC-ROC

In [116]:
probabilities_one_valid = probabilities_valid[:, 1]
auc_roc = roc_auc_score(target_valid, probabilities_one_valid)
auc_roc

0.8441872351955857

Tenemos que el valor de AUC-ROC es de 0.84, en contraste con nuestro valor de F1 que nos dio 0.6 para el conjunto de validación.

**Conclusiones**

Considerando los resultados obtenidos para nuestro dataset, lo más recomendable es que la empresa Beta Bank utilice el Modelo de Bosque aleatorio para explorar si sus clientes dejarán de serlo o se mantendrán. De aquella forma se asegura que lo más fiable lo entregue aquel algoritmo.

Respecto del proyecto en general, fue bastante duro. Aplicar todos los conocimientos de forma consolidada fue un desafío bastante grande y deja un gran sabor respecto de lo que es aplicarlo en un entorno profesional. Espero haber estado correcto en mis presunciones y decisiones.